In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import tqdm
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchvision.models import vgg19
from pathlib import Path
import PIL
from time import time
from PIL import Image

In [ ]:
data_dir = "../input/landmark-recognition-2021"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"
train_file =  "../input/landmark-recognition-2021/train.csv"
sub_file = data_dir +"/sample_submission.csv"

In [ ]:
train_df = pd.read_csv(train_file)
train_df.head(2)

In [ ]:
len(train_df.landmark_id.unique())

In [ ]:
landmark_ids = {lid:i for i, lid in enumerate(train_df.landmark_id.unique())}

In [ ]:
class MakeData(Dataset):
    def __init__(self,csv_file,data_dir,transform = None,data_mode = 'train'):
        super(MakeData,self).__init__()
        self.csv_file = pd.read_csv(csv_file)
        if data_mode == 'train':
            self.landmark_ids = {ids:i for i,ids in enumerate(self.csv_file.landmark_id.unique())}
            self.csv_file['landmark_id'] = self.csv_file['landmark_id'].map(self.landmark_ids)
        
        self.data_dir = data_dir
        self.transform = transform
        
    def __getitem__(self,ids):
        if torch.is_tensor(ids):
            idx = idx.tolist()
        img_id = self.csv_file.iloc[ids, 0]
        img_class = self.csv_file.iloc[ids, 1]
        img_path = os.path.join(self.data_dir, img_id[0], img_id[1], img_id[2], f'{img_id}.jpg')
        img = Image.open(img_path)
        if self.transform is not None:
            img = transform(img)
        sample = [img, img_class, img_id]
        return sample
    def __len__(self):
        return len(self.csv_file)

In [ ]:
transform = transforms.Compose([ transforms.CenterCrop(224), 
                               transforms.ToTensor()])
train_ = MakeData(train_file, train_dir, transform, "train")
test_ = MakeData(sub_file, test_dir, transform, "test")

In [ ]:
valid_size = 0.2
batch_size = 128

In [ ]:
num_train = len(train_)
indices = list(range(num_train))
np.random.seed(100)
np.random.shuffle(indices)
split = int(np.floor(num_train*valid_size))
valid_idx, train_idx = indices[:split], indices[split:]

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_loader = DataLoader(train_, batch_size=batch_size, sampler=train_sampler, num_workers=0)
valid_loader = DataLoader(train_, batch_size=batch_size, sampler=valid_sampler, num_workers=0)
test_loader = DataLoader(test_, batch_size=batch_size, num_workers=0)

In [ ]:
use_cuda = torch.cuda.is_available()
use_cuda

In [ ]:
model_transfer = vgg19(pretrained=True)
for params in model_transfer.features.parameters():
    params.requires_grad=False
in_features = model_transfer.classifier[6].in_features
last_layer = nn.Linear(in_features,  train_df.landmark_id.nunique())
model_transfer.classifier[6] = last_layer
model_transfer.cuda()
print('modelDone')

In [ ]:
model_transfer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_transfer.parameters(), lr=0.001)

In [ ]:
loaders = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

In [ ]:
save_path = "../working/"


In [ ]:
n_epochs = 20

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, num_batch=1, verbose=False):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        start_time = time()
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
#         import pdb; pdb.set_trace()
        for batch_idx, (data, target, img_id) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            train_loss += loss.data.item()*data.size(0)
            if verbose:
                print(f"idx: {batch_idx} Train Loss:{train_loss/(data.size(0) * (batch_idx + 1)) : .6f}")
            if batch_idx > num_batch:
                train_images_used = data.size(0)*(batch_idx + 1)
                break

            
        torch.cuda.empty_cache()
        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target, img_id) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            out = model(data)
            loss = criterion(out, target)
            valid_loss += loss.data.item()*data.size(0) 
            if verbose:
                print(f"idx: {batch_idx} Valid Loss:{valid_loss / (data.size(0) * (batch_idx + 1)) : .6f}")
            if batch_idx > num_batch:
                valid_images_used = data.size(0)*(batch_idx + 1)
                break
        train_loss = train_loss/ train_images_used
        valid_loss = valid_loss / valid_images_used

            
            
        end_time = time()
        time_taken = end_time - start_time
        # print training/validation statistics 
        print('Epoch: {} \t Time: {:.2f} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            time_taken,
            train_loss,
            valid_loss
            ))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        if valid_loss < valid_loss_min:
            if verbose:
                print(f"Valid loss reduced from {valid_loss_min :.6f} to {valid_loss :.6f}, saving model")
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
              
    return model

In [ ]:
def predict(loaders, model, use_cuda, landmark_reverse_map):
    
    
    # set the module to evaluation mode
    model.eval()
    sf = nn.Softmax(dim=1)
    img_id_list = []
    confidence_list = []
    label_list = []
    tot_batch = len(loaders['test'])
    for batch_idx, (data, _, img_id) in enumerate(tqdm.tqdm(loaders['test'])):
        # move to GPU
        if use_cuda:
            data = data.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output = sf(output)
        output = torch.max(output, dim=1)
        confidence = output[0].cpu().detach().numpy()
        label=output[1].cpu().detach().numpy()
        
        img_id_list.extend(list(img_id))
        confidence_list.extend(confidence.tolist())
        label_list.extend(label.tolist())
    
    predict_df = pd.DataFrame({'id': img_id_list, 
                               'landmarks': label_list, 
                               'conf': confidence_list})
    predict_df['landmarks'] = predict_df['landmarks'].map(landmark_reverse_map)
    predict_df['landmarks'] = predict_df['landmarks'].astype(str) +" " + predict_df['conf'].round(6).astype(str)

    predict_df.drop("conf", axis=1, inplace=True)
    return predict_df

In [ ]:
num_epochs = 10
model_learn = train(num_epochs, loaders, model_transfer, optimizer, 
                      loss_fn, use_cuda, os.path.join(save_path,'model_transfer.pt'),num_batch=1000, verbose=False)